<a href="https://colab.research.google.com/github/kikiru328/enterprise_analysis/blob/main/%EA%B0%92df_%EB%9E%AD%ED%81%AC%EB%B6%84%EB%A5%98%2BAbsNet%2B%EA%B3%A0%EA%B0%9D%ED%8A%B9%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pickle5

In [ ]:
import pickle5 as pickle

with open("/content/drive/MyDrive/5조/진짜멘탈터지고 최종/변동률대상_컬럼값모음_df123.pkl",'rb') as f:
  df123 = pickle.load(f)
df123

,고객번호,target,H1_가공식품_r,H2_가공식품_r,H3_가공식품_r,H1_가구/인테리어_r,H2_가구/인테리어_r,H3_가구/인테리어_r,H1_교육/문화용품_r,H2_교육/문화용품_r,H3_교육/문화용품_r,H1_기타_r,H2_기타_r,H3_기타_r,H1_디지털/가전_r,H2_디지털/가전_r,H3_디지털/가전_r,H1_명품_r,H2_명품_r,H3_명품_r,H1_신선식품_r,H2_신선식품_r,H3_신선식품_r,H1_의류_r,H2_의류_r,H3_의류_r,H1_일상용품_r,H2_일상용품_r,H3_일상용품_r,H1_전문스포츠/레저_r,H2_전문스포츠/레저_r,H3_전문스포츠/레저_r,H1_패션잡화_r,H2_패션잡화_r,H3_패션잡화_r,H1_객단가,H2_객단가,H3_객단가,H1_가공식품_구매금액_r,H2_가공식품_구매금액_r,...,H3_명품_app_r,H1_신선식품_app_r,H2_신선식품_app_r,H3_신선식품_app_r,H1_의류_app_r,H2_의류_app_r,H3_의류_app_r,H1_일상용품_app_r,H2_일상용품_app_r,H3_일상용품_app_r,H1_전문스포츠/레저_app_r,H2_전문스포츠/레저_app_r,H3_전문스포츠/레저_app_r,H1_패션잡화_app_r,H2_패션잡화_app_r,H3_패션잡화_app_r,wdys_ratio_1,wdys_ratio_2,wdys_ratio_3,mor_ratio_1,mor_ratio_2,mor_ratio_3,A_ratio_1,A_ratio_2,A_ratio_3,B_ratio_1,B_ratio_2,B_ratio_3,C_ratio_1,C_ratio_2,C_ratio_3,D_ratio_1,D_ratio_2,D_ratio_3,Frequency_1,Frequency_2,Frequency_3,Recency_1,Recency_2,Recency_3
0,6012,1,0.338129,0.198020,0.126016,0.028777,0.004950,0.028455,0.007194,0.004950,0.016260,0.000000,0.000000,0.0,0.003597,0.000000,0.000000,0.000000,0.000000,0.000000,0.039568,0.079208,0.056911,0.442446,0.579208,0.650407,0.104317,0.108911,0.085366,0.010791,0.014851,0.000000,0.025180,0.009901,0.036585,761003.346154,555252.355556,725726.380090,0.006258,0.006724,...,0.000000,0.144991,0.180145,0.364947,0.422322,0.222653,0.264079,0.216588,0.374928,0.279915,0.668941,0.331059,0.000000,0.263670,0.155022,0.517933,0.63,0.50,0.46,0.18,0.15,0.11,0.93,0.91,0.93,0.07,0.07,0.06,0.0,0.02,0.01,0.0,0.00,0.00,121,85,98,179,183,180
1,12446,0,0.225092,0.197674,0.204861,0.129151,0.087209,0.090278,0.029520,0.000000,0.000000,0.000000,0.000000,0.0,0.022140,0.017442,0.020833,0.003690,0.005814,0.000000,0.420664,0.366279,0.472222,0.033210,0.174419,0.045139,0.070111,0.017442,0.090278,0.051661,0.104651,0.062500,0.014760,0.029070,0.013889,206518.899083,164841.886792,171205.734266,0.043831,0.028469,...,0.000000,0.307015,0.203684,0.310607,0.207335,0.309906,0.263612,0.230469,0.375528,0.210916,0.109618,0.187271,0.089474,0.184560,0.366871,0.136060,0.46,0.44,0.64,0.04,0.03,0.02,0.88,1.00,0.91,0.12,0.00,0.04,0.0,0.00,0.01,0.0,0.00,0.05,46,42,59,180,177,176
2,1909,1,0.151442,0.197640,0.157895,0.067308,0.064897,0.062201,0.004808,0.000000,0.002392,0.002404,0.000000,0.0,0.002404,0.002950,0.000000,0.012019,0.000000,0.014354,0.485577,0.471976,0.528708,0.115385,0.100295,0.122010,0.048077,0.064897,0.043062,0.062500,0.058997,0.031100,0.048077,0.038348,0.038278,112765.016529,61794.187135,102602.981132,0.033336,0.084300,...,0.414806,0.269818,0.206753,0.298654,0.373640,0.151045,0.232314,0.183734,0.328961,0.216038,0.296228,0.193697,0.283634,0.158744,0.270742,0.299875,0.78,0.81,0.82,0.01,0.00,0.01,0.71,0.63,0.64,0.29,0.35,0.36,0.0,0.02,0.00,0.0,0.00,0.00,71,61,70,178,182,177
3,9627,1,0.383648,0.415929,0.320513,0.025157,0.053097,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.006289,0.000000,0.006410,0.012579,0.008850,0.006410,0.163522,0.150442,0.326923,0.188679,0.176991,0.147436,0.125786,0.150442,0.083333,0.037736,0.017699,0.044872,0.056604,0.026549,0.051282,59783.605442,47599.230769,70738.000000,0.120878,0.125164,...,0.020435,0.285130,0.276717,0.310047,0.172181,0.131866,0.273806,0.204997,0.348915,0.321172,0.176654,0.331173,0.245805,0.079018,0.168423,0.105193,0.72,0.65,0.78,0.05,0.08,0.03,0.97,1.00,1.00,0.03,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,67,49,62,180,180,176
4,16297,1,0.195620,0.175676,0.238095,0.032117,0.011583,0.004762,0.013139,0.025097,0.028571,0.000000,0.003861,0.0,0.001460,0.000000,0.002381,0.000000,0.000000,0.007143,0.613139,0.640927,0.469048,0.054015,0.046332,0.061905,0.070073,0.084942,0.154762,0.008759,0.000000,0.014286,0.011679,0.011583,0.019048,115660.355301,112482.666667,184490.592965,0.024133,0.022075,...,0.648835,0.206689,0.245712,0.273585,0.191062,0.196134,0.269730,0.216715,0.199403,0.287080,0.862520,0.000000,0.137480,0.234598,0.251835,0.353934,0.69,0.70,0.59,0.40,0.29,0.14,0.33,0.32,0.30,0.67,0.68,0.70,0.0,0.00,0.00,0.0,0.00,0.00,145,128,110,180,183,180
...

In [ ]:
# 랭크가 낮을수록 감소하는 녀석들임

def grade_to(x):
    if x <= 19085*0.1:
        return 1
    elif x <= 19085*0.2:
        return 2
    elif x <= 19085*0.3:
        return 3
    elif x <= 19085*0.4:
        return 4
    elif x <= 19085*0.5:
        return 5
    elif x <= 19085*0.6:
        return 6
    elif x <= 19085*0.7:
        return 7
    elif x <= 19085*0.8:
        return 8
    elif x <= 19085*0.9:
        return 9
    else:
        return 10
    
def rank_df(df_name, columns):
    return df_name.rank()[columns].apply(grade_to)

from tqdm import tqdm

change_list = df123.columns[2:]
for i in tqdm(change_list):
    df123[i] = rank_df(df123, i)

df123

100%|██████████| 126/126 [00:25<00:00,  4.91it/s]


,고객번호,target,H1_가공식품_r,H2_가공식품_r,H3_가공식품_r,H1_가구/인테리어_r,H2_가구/인테리어_r,H3_가구/인테리어_r,H1_교육/문화용품_r,H2_교육/문화용품_r,H3_교육/문화용품_r,H1_기타_r,H2_기타_r,H3_기타_r,H1_디지털/가전_r,H2_디지털/가전_r,H3_디지털/가전_r,H1_명품_r,H2_명품_r,H3_명품_r,H1_신선식품_r,H2_신선식품_r,H3_신선식품_r,H1_의류_r,H2_의류_r,H3_의류_r,H1_일상용품_r,H2_일상용품_r,H3_일상용품_r,H1_전문스포츠/레저_r,H2_전문스포츠/레저_r,H3_전문스포츠/레저_r,H1_패션잡화_r,H2_패션잡화_r,H3_패션잡화_r,H1_객단가,H2_객단가,H3_객단가,H1_가공식품_구매금액_r,H2_가공식품_구매금액_r,...,H3_명품_app_r,H1_신선식품_app_r,H2_신선식품_app_r,H3_신선식품_app_r,H1_의류_app_r,H2_의류_app_r,H3_의류_app_r,H1_일상용품_app_r,H2_일상용품_app_r,H3_일상용품_app_r,H1_전문스포츠/레저_app_r,H2_전문스포츠/레저_app_r,H3_전문스포츠/레저_app_r,H1_패션잡화_app_r,H2_패션잡화_app_r,H3_패션잡화_app_r,wdys_ratio_1,wdys_ratio_2,wdys_ratio_3,mor_ratio_1,mor_ratio_2,mor_ratio_3,A_ratio_1,A_ratio_2,A_ratio_3,B_ratio_1,B_ratio_2,B_ratio_3,C_ratio_1,C_ratio_2,C_ratio_3,D_ratio_1,D_ratio_2,D_ratio_3,Frequency_1,Frequency_2,Frequency_3,Recency_1,Recency_2,Recency_3
0,6012,1,4,1,1,7,5,7,8,7,9,4,4,4,8,4,4,4,4,4,1,1,1,10,10,10,5,5,4,6,7,2,8,6,9,10,10,10,1,1,...,4,1,1,10,9,5,7,4,10,8,10,8,2,7,5,9,4,2,1,10,9,8,9,9,9,4,4,4,3,6,5,5,5,5,10,6,8,5,8,8
1,12446,0,1,1,1,10,10,10,10,3,3,4,4,4,10,10,10,9,9,4,9,8,9,6,10,7,3,1,4,9,10,10,7,8,7,10,10,10,2,1,...,4,9,2,10,5,8,7,5,10,3,4,5,4,5,8,4,1,1,4,5,4,3,9,10,9,5,2,4,3,3,5,5,5,10,2,1,3,8,1,2
2,1909,1,1,1,1,9,9,9,7,3,6,7,4,4,8,8,4,10,4,10,9,9,10,9,9,9,2,3,2,10,10,9,9,9,9,9,8,9,1,3,...,9,8,2,9,9,3,6,2,9,4,8,5,7,5,7,8,9,9,10,2,1,2,8,8,8,6,6,6,3,6,3,5,5,5,5,3,4,3,5,2
3,9627,1,5,5,3,7,8,6,3,3,3,4,4,4,9,4,9,10,9,9,2,2,6,10,10,9,6,7,4,9,8,9,9,8,9,8,7,8,4,4,...,8,9,8,10,4,3,7,3,10,9,5,8,7,4,5,4,7,4,9,6,7,4,9,10,10,4,2,2,3,3,3,5,5,5,5,2,3,8,2,2
4,16297,1,1,1,1,7,6,5,9,10,10,4,8,4,7,4,8,4,4,9,10,10,9,7,7,8,3,4,8,6,2,7,6,6,7,9,9,10,1,1,...,10,3,5,8,5,4,7,4,3,8,10,2,4,6,7,8,6,6,3,10,10,9,6,6,6,7,7,7,3,3,3,5,5,5,10,10,9,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19080,19355,0,4,8,9,2,6,2,3,3,3,4,8,4,4,4,4,4,4,4,1,1,1,1,6,1,10,10,10,2,7,2,2,2,2,4,3,2,3,5,...,4,1,1,10,1,10,1,6,7,6,2,9,2,2,2,2,1,7,2,7,8,9,2,6,2,2,4,5,3,3,3,10,10,10,1,3,3,3,8,3
19081,18986,0,1,3,1,2,5,2,3,3,7,4,4,4,4,4,4,4,4,4,1,1,1,1,7,4,10,10,10,2,2,2,2,5,2,6,7,6,2,2,...,4,1,10,10,1,9,9,6,10,4,2,2,2,2,10,2,3,6,9,10,10,8,2,6,5,2,6,5,3,6,3,10,10,10,1,2,2,3,1,8
19082,6868,0,1,1,1,2,2,2,3,3,3,4,4,4,4,4,4,4,4,4,1,1,1,1,1,1,10,10,10,2,2,2,2,2,2,9,6,4,1,1,...,4,1,1,10,1,1,1,9,3,2,2,2,2,2,2,2,10,2,1,1,1,5,2,2,5,2,2,2,3,5,3,10,10,10,1,1,1,1,1,1
19083,18977,0,8,3,5,2,2,2,3,3,3,4,4,4,4,4,4,4,4,4,1,1,1,1,4,5,10,10,10,2,2,2,2,2,2,1,2,2,4,2,...,4,1,1,1,1,10,9,6,7,6,2,2,2,2,2,2,3,7,10,7,10,10,2,4,5,2,2,2,3,3,3,10,10,10,1,3,3,1,5,8


In [ ]:
abs_net_df = df123
고객번호추가 = abs_net_df['고객번호']
target추가 = abs_net_df['target']
abs_net_df = abs_net_df.drop(['고객번호','target'],axis=1)

import pandas as pd
import numpy as np
changed_df = pd.DataFrame()
changed_df

group_col =[]
for i in range(int(len(abs_net_df.columns)/3)):
  group_col.append(list(abs_net_df.columns[i*3:i*3+3]))

columns = []
for cols in group_col:
  if cols[0][0]=='H':
    colname = cols[0][3:]
    columns.append(colname)
  else:
    colname = cols[0][:-2]
    columns.append(colname)

for index,cols in enumerate(group_col):
  get_cols = abs_net_df[cols]
  col1=get_cols.iloc[:,0]
  col2=get_cols.iloc[:,1]
  col3=get_cols.iloc[:,2]
  NetDiff = col3-col1
  AbsDiffsum = np.abs(col2-col1) + np.abs(col3-col2)
  changed_df[f'{columns[index]}_Net'] = NetDiff
  changed_df[f'{columns[index]}_Abs'] = AbsDiffsum

changed_df['고객번호'] = 고객번호추가
changed_df['target'] = target추가



In [ ]:
changed_df.head()

,가공식품_r_Net,가공식품_r_Abs,가구/인테리어_r_Net,가구/인테리어_r_Abs,교육/문화용품_r_Net,교육/문화용품_r_Abs,기타_r_Net,기타_r_Abs,디지털/가전_r_Net,디지털/가전_r_Abs,명품_r_Net,명품_r_Abs,신선식품_r_Net,신선식품_r_Abs,의류_r_Net,의류_r_Abs,일상용품_r_Net,일상용품_r_Abs,전문스포츠/레저_r_Net,전문스포츠/레저_r_Abs,패션잡화_r_Net,패션잡화_r_Abs,객단가_Net,객단가_Abs,가공식품_구매금액_r_Net,가공식품_구매금액_r_Abs,가구/인테리어_구매금액_r_Net,가구/인테리어_구매금액_r_Abs,교육/문화용품_구매금액_r_Net,교육/문화용품_구매금액_r_Abs,기타_구매금액_r_Net,기타_구매금액_r_Abs,디지털/가전_구매금액_r_Net,디지털/가전_구매금액_r_Abs,명품_구매금액_r_Net,명품_구매금액_r_Abs,신선식품_구매금액_r_Net,신선식품_구매금액_r_Abs,의류_구매금액_r_Net,의류_구매금액_r_Abs,...,가공식품_app_r_Net,가공식품_app_r_Abs,가구/인테리어_app_r_Net,가구/인테리어_app_r_Abs,교육/문화용품_app_r_Net,교육/문화용품_app_r_Abs,기타_app_r_Net,기타_app_r_Abs,디지털/가전_app_r_Net,디지털/가전_app_r_Abs,명품_app_r_Net,명품_app_r_Abs,신선식품_app_r_Net,신선식품_app_r_Abs,의류_app_r_Net,의류_app_r_Abs,일상용품_app_r_Net,일상용품_app_r_Abs,전문스포츠/레저_app_r_Net,전문스포츠/레저_app_r_Abs,패션잡화_app_r_Net,패션잡화_app_r_Abs,wdys_ratio_Net,wdys_ratio_Abs,mor_ratio_Net,mor_ratio_Abs,A_ratio_Net,A_ratio_Abs,B_ratio_Net,B_ratio_Abs,C_ratio_Net,C_ratio_Abs,D_ratio_Net,D_ratio_Abs,Frequency_Net,Frequency_Abs,Recency_Net,Recency_Abs,고객번호,target
0,-3,3,0,4,1,3,0,0,-4,4,0,0,0,0,0,0,-1,1,-4,6,1,5,0,0,0,0,1,1,-1,1,0,0,-3,3,0,0,0,0,0,0,...,4,6,4,8,-1,5,0,0,-5,5,0,0,9,9,-2,6,4,8,-8,8,2,6,-3,3,-2,2,0,0,0,0,2,4,0,0,-2,6,3,3,6012,1
1,0,0,0,0,-7,7,0,0,0,0,-5,5,0,2,1,7,1,5,1,1,0,2,0,0,-1,1,1,1,-5,5,0,0,0,2,-5,5,1,3,1,9,...,-3,3,3,3,-7,7,0,0,1,3,-4,4,1,15,2,4,-2,12,0,2,-1,7,3,3,-2,2,0,2,-1,5,2,2,5,5,1,3,-6,8,12446,0
2,0,0,0,0,-1,7,-3,3,-4,4,0,12,1,1,0,0,0,2,-1,1,0,0,0,2,0,4,1,1,-1,7,-3,3,-4,8,0,12,1,1,0,2,...,-6,6,0,6,2,10,-6,6,-4,8,-1,11,1,13,-3,9,2,12,-1,5,3,3,1,1,0,2,0,0,0,0,0,6,0,0,-1,3,-1,5,1909,1
3,-2,2,-1,3,0,0,0,0,0,10,-1,1,4,4,-1,1,-2,4,0,2,0,2,0,2,-2,2,-2,6,0,0,0,0,2,10,-2,2,2,2,1,1,...,-6,6,-1,1,0,0,0,0,3,9,-2,2,1,3,3,5,6,8,2,4,0,2,2,8,-2,4,1,1,-2,2,0,0,0,0,-2,4,-6,6,9627,1
4,0,0,-2,2,1,1,0,8,1,7,5,5,-1,1,1,1,5,5,1,9,1,1,1,1,0,0,-1,3,0,0,0,6,1,7,6,6,-2,2,0,0,...,-2,2,0,10,2,2,0,10,1,9,6,6,5,5,2,4,4,6,-6,10,2,2,-3,3,-1,1,0,0,0,0,0,0,0,0,-1,1,0,0,16297,1


In [ ]:
import pickle5 as pickle
with open('/content/drive/MyDrive/5조/진짜진짜최종/2차데이터123분기.pkl', 'rb') as f:
    cust = pickle.load(f)

changed_df = pd.merge(changed_df,cust[['고객번호', '성별', '연령대', '거주지역']],on='고객번호', how='left')
changed_df.head()

,가공식품_r_Net,가공식품_r_Abs,가구/인테리어_r_Net,가구/인테리어_r_Abs,교육/문화용품_r_Net,교육/문화용품_r_Abs,기타_r_Net,기타_r_Abs,디지털/가전_r_Net,디지털/가전_r_Abs,명품_r_Net,명품_r_Abs,신선식품_r_Net,신선식품_r_Abs,의류_r_Net,의류_r_Abs,일상용품_r_Net,일상용품_r_Abs,전문스포츠/레저_r_Net,전문스포츠/레저_r_Abs,패션잡화_r_Net,패션잡화_r_Abs,객단가_Net,객단가_Abs,가공식품_구매금액_r_Net,가공식품_구매금액_r_Abs,가구/인테리어_구매금액_r_Net,가구/인테리어_구매금액_r_Abs,교육/문화용품_구매금액_r_Net,교육/문화용품_구매금액_r_Abs,기타_구매금액_r_Net,기타_구매금액_r_Abs,디지털/가전_구매금액_r_Net,디지털/가전_구매금액_r_Abs,명품_구매금액_r_Net,명품_구매금액_r_Abs,신선식품_구매금액_r_Net,신선식품_구매금액_r_Abs,의류_구매금액_r_Net,의류_구매금액_r_Abs,...,가구/인테리어_app_r_Abs,교육/문화용품_app_r_Net,교육/문화용품_app_r_Abs,기타_app_r_Net,기타_app_r_Abs,디지털/가전_app_r_Net,디지털/가전_app_r_Abs,명품_app_r_Net,명품_app_r_Abs,신선식품_app_r_Net,신선식품_app_r_Abs,의류_app_r_Net,의류_app_r_Abs,일상용품_app_r_Net,일상용품_app_r_Abs,전문스포츠/레저_app_r_Net,전문스포츠/레저_app_r_Abs,패션잡화_app_r_Net,패션잡화_app_r_Abs,wdys_ratio_Net,wdys_ratio_Abs,mor_ratio_Net,mor_ratio_Abs,A_ratio_Net,A_ratio_Abs,B_ratio_Net,B_ratio_Abs,C_ratio_Net,C_ratio_Abs,D_ratio_Net,D_ratio_Abs,Frequency_Net,Frequency_Abs,Recency_Net,Recency_Abs,고객번호,target,성별,연령대,거주지역
0,-3,3,0,4,1,3,0,0,-4,4,0,0,0,0,0,0,-1,1,-4,6,1,5,0,0,0,0,1,1,-1,1,0,0,-3,3,0,0,0,0,0,0,...,8,-1,5,0,0,-5,5,0,0,9,9,-2,6,4,8,-8,8,2,6,-3,3,-2,2,0,0,0,0,2,4,0,0,-2,6,3,3,6012,1,F,40세~44세,서울 동작구
1,0,0,0,0,-7,7,0,0,0,0,-5,5,0,2,1,7,1,5,1,1,0,2,0,0,-1,1,1,1,-5,5,0,0,0,2,-5,5,1,3,1,9,...,3,-7,7,0,0,1,3,-4,4,1,15,2,4,-2,12,0,2,-1,7,3,3,-2,2,0,2,-1,5,2,2,5,5,1,3,-6,8,12446,0,F,45세~49세,서울 광진구
2,0,0,0,0,-1,7,-3,3,-4,4,0,12,1,1,0,0,0,2,-1,1,0,0,0,2,0,4,1,1,-1,7,-3,3,-4,8,0,12,1,1,0,2,...,6,2,10,-6,6,-4,8,-1,11,1,13,-3,9,2,12,-1,5,3,3,1,1,0,2,0,0,0,0,0,6,0,0,-1,3,-1,5,1909,1,F,50세~54세,경기 김포시
3,-2,2,-1,3,0,0,0,0,0,10,-1,1,4,4,-1,1,-2,4,0,2,0,2,0,2,-2,2,-2,6,0,0,0,0,2,10,-2,2,2,2,1,1,...,1,0,0,0,0,3,9,-2,2,1,3,3,5,6,8,2,4,0,2,2,8,-2,4,1,1,-2,2,0,0,0,0,-2,4,-6,6,9627,1,F,30세~34세,부산 기장
4,0,0,-2,2,1,1,0,8,1,7,5,5,-1,1,1,1,5,5,1,9,1,1,1,1,0,0,-1,3,0,0,0,6,1,7,6,6,-2,2,0,0,...,10,2,2,0,10,1,9,6,6,5,5,2,4,4,6,-6,10,2,2,-3,3,-1,1,0,0,0,0,0,0,0,0,-1,1,0,0,16297,1,F,45세~49세,서울 도봉구


In [ ]:
import pickle5 as pickle
with open("/content/drive/MyDrive/5조/진짜진짜최종/123_최종추가_컬럼/df123_멤버십.pkl", 'rb') as f:
    membership= pickle.load(f)
membership = membership.reset_index()
changed_df = pd.merge(changed_df,membership[['고객번호','abs_13']],on='고객번호', how='left')
changed_df['abs_13'].fillna(0,inplace=True)
changed_df = changed_df.rename(columns={'abs_13':'membership_abs'})
changed_df.head()

,가공식품_r_Net,가공식품_r_Abs,가구/인테리어_r_Net,가구/인테리어_r_Abs,교육/문화용품_r_Net,교육/문화용품_r_Abs,기타_r_Net,기타_r_Abs,디지털/가전_r_Net,디지털/가전_r_Abs,명품_r_Net,명품_r_Abs,신선식품_r_Net,신선식품_r_Abs,의류_r_Net,의류_r_Abs,일상용품_r_Net,일상용품_r_Abs,전문스포츠/레저_r_Net,전문스포츠/레저_r_Abs,패션잡화_r_Net,패션잡화_r_Abs,객단가_Net,객단가_Abs,가공식품_구매금액_r_Net,가공식품_구매금액_r_Abs,가구/인테리어_구매금액_r_Net,가구/인테리어_구매금액_r_Abs,교육/문화용품_구매금액_r_Net,교육/문화용품_구매금액_r_Abs,기타_구매금액_r_Net,기타_구매금액_r_Abs,디지털/가전_구매금액_r_Net,디지털/가전_구매금액_r_Abs,명품_구매금액_r_Net,명품_구매금액_r_Abs,신선식품_구매금액_r_Net,신선식품_구매금액_r_Abs,의류_구매금액_r_Net,의류_구매금액_r_Abs,...,교육/문화용품_app_r_Net,교육/문화용품_app_r_Abs,기타_app_r_Net,기타_app_r_Abs,디지털/가전_app_r_Net,디지털/가전_app_r_Abs,명품_app_r_Net,명품_app_r_Abs,신선식품_app_r_Net,신선식품_app_r_Abs,의류_app_r_Net,의류_app_r_Abs,일상용품_app_r_Net,일상용품_app_r_Abs,전문스포츠/레저_app_r_Net,전문스포츠/레저_app_r_Abs,패션잡화_app_r_Net,패션잡화_app_r_Abs,wdys_ratio_Net,wdys_ratio_Abs,mor_ratio_Net,mor_ratio_Abs,A_ratio_Net,A_ratio_Abs,B_ratio_Net,B_ratio_Abs,C_ratio_Net,C_ratio_Abs,D_ratio_Net,D_ratio_Abs,Frequency_Net,Frequency_Abs,Recency_Net,Recency_Abs,고객번호,target,성별,연령대,거주지역,membership_abs
0,-3,3,0,4,1,3,0,0,-4,4,0,0,0,0,0,0,-1,1,-4,6,1,5,0,0,0,0,1,1,-1,1,0,0,-3,3,0,0,0,0,0,0,...,-1,5,0,0,-5,5,0,0,9,9,-2,6,4,8,-8,8,2,6,-3,3,-2,2,0,0,0,0,2,4,0,0,-2,6,3,3,6012,1,F,40세~44세,서울 동작구,0.0
1,0,0,0,0,-7,7,0,0,0,0,-5,5,0,2,1,7,1,5,1,1,0,2,0,0,-1,1,1,1,-5,5,0,0,0,2,-5,5,1,3,1,9,...,-7,7,0,0,1,3,-4,4,1,15,2,4,-2,12,0,2,-1,7,3,3,-2,2,0,2,-1,5,2,2,5,5,1,3,-6,8,12446,0,F,45세~49세,서울 광진구,0.0
2,0,0,0,0,-1,7,-3,3,-4,4,0,12,1,1,0,0,0,2,-1,1,0,0,0,2,0,4,1,1,-1,7,-3,3,-4,8,0,12,1,1,0,2,...,2,10,-6,6,-4,8,-1,11,1,13,-3,9,2,12,-1,5,3,3,1,1,0,2,0,0,0,0,0,6,0,0,-1,3,-1,5,1909,1,F,50세~54세,경기 김포시,0.0
3,-2,2,-1,3,0,0,0,0,0,10,-1,1,4,4,-1,1,-2,4,0,2,0,2,0,2,-2,2,-2,6,0,0,0,0,2,10,-2,2,2,2,1,1,...,0,0,0,0,3,9,-2,2,1,3,3,5,6,8,2,4,0,2,2,8,-2,4,1,1,-2,2,0,0,0,0,-2,4,-6,6,9627,1,F,30세~34세,부산 기장,0.0
4,0,0,-2,2,1,1,0,8,1,7,5,5,-1,1,1,1,5,5,1,9,1,1,1,1,0,0,-1,3,0,0,0,6,1,7,6,6,-2,2,0,0,...,2,2,0,10,1,9,6,6,5,5,2,4,4,6,-6,10,2,2,-3,3,-1,1,0,0,0,0,0,0,0,0,-1,1,0,0,16297,1,F,45세~49세,서울 도봉구,0.0


In [ ]:
changed_df = changed_df[['성별', '연령대', '거주지역', 'membership_abs', '객단가_Net', '객단가_Abs',
       '가공식품_app_r_Net', '가공식품_app_r_Abs', '가구/인테리어_app_r_Net',
       '가구/인테리어_app_r_Abs', '교육/문화용품_app_r_Net', '교육/문화용품_app_r_Abs',
       '기타_app_r_Net', '기타_app_r_Abs', '디지털/가전_app_r_Net', '디지털/가전_app_r_Abs',
       '명품_app_r_Net', '명품_app_r_Abs', '신선식품_app_r_Net', '신선식품_app_r_Abs',
       '의류_app_r_Net', '의류_app_r_Abs', '일상용품_app_r_Net', '일상용품_app_r_Abs',
       '전문스포츠/레저_app_r_Net', '전문스포츠/레저_app_r_Abs', '패션잡화_app_r_Net',
       '패션잡화_app_r_Abs', 'wdys_ratio_Net', 'wdys_ratio_Abs', 'mor_ratio_Net',
       'mor_ratio_Abs', 'A_ratio_Net', 'A_ratio_Abs', 'B_ratio_Net',
       'B_ratio_Abs', 'C_ratio_Net', 'C_ratio_Abs', 'D_ratio_Net',
       'D_ratio_Abs', 'Frequency_Net', 'Frequency_Abs', 'Recency_Net',
       'Recency_Abs', 'target']]

In [ ]:
changed_df.head()

,고객번호,성별,연령대,거주지역,membership_abs,객단가_Net,객단가_Abs,가공식품_r_Net,가공식품_r_Abs,가구/인테리어_r_Net,가구/인테리어_r_Abs,교육/문화용품_r_Net,교육/문화용품_r_Abs,기타_r_Net,기타_r_Abs,디지털/가전_r_Net,디지털/가전_r_Abs,명품_r_Net,명품_r_Abs,신선식품_r_Net,신선식품_r_Abs,의류_r_Net,의류_r_Abs,일상용품_r_Net,일상용품_r_Abs,전문스포츠/레저_r_Net,전문스포츠/레저_r_Abs,패션잡화_r_Net,패션잡화_r_Abs,가공식품_구매금액_r_Net,가공식품_구매금액_r_Abs,가구/인테리어_구매금액_r_Net,가구/인테리어_구매금액_r_Abs,교육/문화용품_구매금액_r_Net,교육/문화용품_구매금액_r_Abs,기타_구매금액_r_Net,기타_구매금액_r_Abs,디지털/가전_구매금액_r_Net,디지털/가전_구매금액_r_Abs,명품_구매금액_r_Net,...,패션잡화_구매금액_r_Abs,가공식품_app_r_Net,가공식품_app_r_Abs,가구/인테리어_app_r_Net,가구/인테리어_app_r_Abs,교육/문화용품_app_r_Net,교육/문화용품_app_r_Abs,기타_app_r_Net,기타_app_r_Abs,디지털/가전_app_r_Net,디지털/가전_app_r_Abs,명품_app_r_Net,명품_app_r_Abs,신선식품_app_r_Net,신선식품_app_r_Abs,의류_app_r_Net,의류_app_r_Abs,일상용품_app_r_Net,일상용품_app_r_Abs,전문스포츠/레저_app_r_Net,전문스포츠/레저_app_r_Abs,패션잡화_app_r_Net,패션잡화_app_r_Abs,wdys_ratio_Net,wdys_ratio_Abs,mor_ratio_Net,mor_ratio_Abs,A_ratio_Net,A_ratio_Abs,B_ratio_Net,B_ratio_Abs,C_ratio_Net,C_ratio_Abs,D_ratio_Net,D_ratio_Abs,Frequency_Net,Frequency_Abs,Recency_Net,Recency_Abs,target
0,6012,F,40세~44세,서울 동작구,0.0,0,0,-3,3,0,4,1,3,0,0,-4,4,0,0,0,0,0,0,-1,1,-4,6,1,5,0,0,1,1,-1,1,0,0,-3,3,0,...,1,4,6,4,8,-1,5,0,0,-5,5,0,0,9,9,-2,6,4,8,-8,8,2,6,-3,3,-2,2,0,0,0,0,2,4,0,0,-2,6,3,3,1
1,12446,F,45세~49세,서울 광진구,0.0,0,0,0,0,0,0,-7,7,0,0,0,0,-5,5,0,2,1,7,1,5,1,1,0,2,-1,1,1,1,-5,5,0,0,0,2,-5,...,5,-3,3,3,3,-7,7,0,0,1,3,-4,4,1,15,2,4,-2,12,0,2,-1,7,3,3,-2,2,0,2,-1,5,2,2,5,5,1,3,-6,8,0
2,1909,F,50세~54세,경기 김포시,0.0,0,2,0,0,0,0,-1,7,-3,3,-4,4,0,12,1,1,0,0,0,2,-1,1,0,0,0,4,1,1,-1,7,-3,3,-4,8,0,...,4,-6,6,0,6,2,10,-6,6,-4,8,-1,11,1,13,-3,9,2,12,-1,5,3,3,1,1,0,2,0,0,0,0,0,6,0,0,-1,3,-1,5,1
3,9627,F,30세~34세,부산 기장,0.0,0,2,-2,2,-1,3,0,0,0,0,0,10,-1,1,4,4,-1,1,-2,4,0,2,0,2,-2,2,-2,6,0,0,0,0,2,10,-2,...,1,-6,6,-1,1,0,0,0,0,3,9,-2,2,1,3,3,5,6,8,2,4,0,2,2,8,-2,4,1,1,-2,2,0,0,0,0,-2,4,-6,6,1
4,16297,F,45세~49세,서울 도봉구,0.0,1,1,0,0,-2,2,1,1,0,8,1,7,5,5,-1,1,1,1,5,5,1,9,1,1,0,0,-1,3,0,0,0,6,1,7,6,...,1,-2,2,0,10,2,2,0,10,1,9,6,6,5,5,2,4,4,6,-6,10,2,2,-3,3,-1,1,0,0,0,0,0,0,0,0,-1,1,0,0,1


In [ ]:
from sklearn.preprocessing import LabelEncoder
                                                                                                                                                        
train_df = changed_df.drop('고객번호',axis = 1)

encoder = LabelEncoder()


train_df['성별'] = encoder.fit_transform(train_df['성별'])
train_df['연령대'] = encoder.fit_transform(train_df['연령대'])
train_df['거주지역']=train_df['거주지역'].fillna('불명')
train_df['거주지역'] = encoder.fit_transform(train_df['거주지역'])

In [ ]:
with open("/content/drive/MyDrive/5조/진짜멘탈터지고 최종/df_ml_123.pkl",'wb') as test_save:
  pickle.dump(train_df, test_save)

In [ ]:
import pickle5 as pickle

with open("/content/drive/MyDrive/5조/진짜멘탈터지고 최종/변동률대상_컬럼값모음_df1234.pkl",'rb') as f:
  df1234 = pickle.load(f)
df1234

,고객번호,target,H1_가공식품_r,H2_가공식품_r,H3_가공식품_r,H4_가공식품_r,H1_가구/인테리어_r,H2_가구/인테리어_r,H3_가구/인테리어_r,H4_가구/인테리어_r,H1_교육/문화용품_r,H2_교육/문화용품_r,H3_교육/문화용품_r,H4_교육/문화용품_r,H1_기타_r,H2_기타_r,H3_기타_r,H4_기타_r,H1_디지털/가전_r,H2_디지털/가전_r,H3_디지털/가전_r,H4_디지털/가전_r,H1_명품_r,H2_명품_r,H3_명품_r,H4_명품_r,H1_신선식품_r,H2_신선식품_r,H3_신선식품_r,H4_신선식품_r,H1_의류_r,H2_의류_r,H3_의류_r,H4_의류_r,H1_일상용품_r,H2_일상용품_r,H3_일상용품_r,H4_일상용품_r,H1_전문스포츠/레저_r,H2_전문스포츠/레저_r,...,H1_전문스포츠/레저_app_r,H2_전문스포츠/레저_app_r,H3_전문스포츠/레저_app_r,H4_전문스포츠/레저_app_r,H1_패션잡화_app_r,H2_패션잡화_app_r,H3_패션잡화_app_r,H4_패션잡화_app_r,wdys_ratio_1,wdys_ratio_2,wdys_ratio_3,wdys_ratio_4,mor_ratio_1,mor_ratio_2,mor_ratio_3,mor_ratio_4,A_ratio_1,A_ratio_2,A_ratio_3,A_ratio_4,B_ratio_1,B_ratio_2,B_ratio_3,B_ratio_4,C_ratio_1,C_ratio_2,C_ratio_3,C_ratio_4,D_ratio_1,D_ratio_2,D_ratio_3,D_ratio_4,Frequency_1,Frequency_2,Frequency_3,Frequency_4,Recency_1,Recency_2,Recency_3,Recency_4
0,6012,1,0.338129,0.198020,0.126016,0.299435,0.028777,0.004950,0.028455,0.028249,0.007194,0.004950,0.016260,0.033898,0.000000,0.000000,0.0,0.000000,0.003597,0.000000,0.000000,0.005650,0.000000,0.000000,0.000000,0.000000,0.039568,0.079208,0.056911,0.084746,0.442446,0.579208,0.650407,0.367232,0.104317,0.108911,0.085366,0.169492,0.010791,0.014851,...,0.668941,0.331059,0.000000,0.000000,0.263670,0.155022,0.517933,0.063374,0.63,0.50,0.46,0.58,0.18,0.15,0.11,0.06,0.93,0.91,0.93,0.81,0.07,0.07,0.06,0.19,0.0,0.02,0.01,0.00,0.0,0.00,0.00,0.00,121,85,98,84,179,183,180,183
1,12446,0,0.225092,0.197674,0.204861,0.317708,0.129151,0.087209,0.090278,0.093750,0.029520,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.022140,0.017442,0.020833,0.005208,0.003690,0.005814,0.000000,0.005208,0.420664,0.366279,0.472222,0.333333,0.033210,0.174419,0.045139,0.067708,0.070111,0.017442,0.090278,0.046875,0.051661,0.104651,...,0.109618,0.187271,0.089474,0.613637,0.184560,0.366871,0.136060,0.312509,0.46,0.44,0.64,0.66,0.04,0.03,0.02,0.01,0.88,1.00,0.91,0.89,0.12,0.00,0.04,0.08,0.0,0.00,0.01,0.02,0.0,0.00,0.05,0.02,46,42,59,38,180,177,176,181
2,1909,1,0.151442,0.197640,0.157895,0.211302,0.067308,0.064897,0.062201,0.063882,0.004808,0.000000,0.002392,0.002457,0.002404,0.000000,0.0,0.000000,0.002404,0.002950,0.000000,0.000000,0.012019,0.000000,0.014354,0.000000,0.485577,0.471976,0.528708,0.503686,0.115385,0.100295,0.122010,0.061425,0.048077,0.064897,0.043062,0.058968,0.062500,0.058997,...,0.296228,0.193697,0.283634,0.226440,0.158744,0.270742,0.299875,0.270640,0.78,0.81,0.82,0.76,0.01,0.00,0.01,0.04,0.71,0.63,0.64,0.58,0.29,0.35,0.36,0.42,0.0,0.02,0.00,0.00,0.0,0.00,0.00,0.00,71,61,70,79,178,182,177,175
3,9627,0,0.383648,0.415929,0.320513,0.516484,0.025157,0.053097,0.012821,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.006289,0.000000,0.006410,0.043956,0.012579,0.008850,0.006410,0.000000,0.163522,0.150442,0.326923,0.076923,0.188679,0.176991,0.147436,0.076923,0.125786,0.150442,0.083333,0.076923,0.037736,0.017699,...,0.176654,0.331173,0.245805,0.246368,0.079018,0.168423,0.105193,0.647366,0.72,0.65,0.78,0.42,0.05,0.08,0.03,0.09,0.97,1.00,1.00,0.63,0.03,0.00,0.00,0.01,0.0,0.00,0.00,0.36,0.0,0.00,0.00,0.00,67,49,62,34,180,180,176,172
4,16297,1,0.195620,0.175676,0.238095,0.274143,0.032117,0.011583,0.004762,0.015576,0.013139,0.025097,0.028571,0.024922,0.000000,0.003861,0.0,0.003115,0.001460,0.000000,0.002381,0.003115,0.000000,0.000000,0.007143,0.012461,0.613139,0.640927,0.469048,0.383178,0.054015,0.046332,0.061905,0.084112,0.070073,0.084942,0.154762,0.180685,0.008759,0.000000,...,0.862520,0.000000,0.137480,0.000000,0.234598,0.251835,0.353934,0.159633,0.69,0.70,0.59,0.79,0.40,0.29,0.14,0.12,0.33,0.32,0.30,0.33,0.67,0.68,0.70,0.66,0.0,0.00,0.00,0.01,0.0,0.00,0.00,0.00,145,128,110,91,180,183,180,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
# 랭크가 낮을수록 감소하는 녀석들임

def grade_to(x):
    if x <= 19085*0.1:
        return 1
    elif x <= 19085*0.2:
        return 2
    elif x <= 19085*0.3:
        return 3
    elif x <= 19085*0.4:
        return 4
    elif x <= 19085*0.5:
        return 5
    elif x <= 19085*0.6:
        return 6
    elif x <= 19085*0.7:
        return 7
    elif x <= 19085*0.8:
        return 8
    elif x <= 19085*0.9:
        return 9
    else:
        return 10
    
def rank_df(df_name, columns):
    return df_name.rank()[columns].apply(grade_to)

from tqdm import tqdm

change_list = df1234.columns[2:]
for i in tqdm(change_list):
    df1234[i] = rank_df(df1234, i)

df1234

100%|██████████| 168/168 [00:44<00:00,  3.76it/s]


,고객번호,target,H1_가공식품_r,H2_가공식품_r,H3_가공식품_r,H4_가공식품_r,H1_가구/인테리어_r,H2_가구/인테리어_r,H3_가구/인테리어_r,H4_가구/인테리어_r,H1_교육/문화용품_r,H2_교육/문화용품_r,H3_교육/문화용품_r,H4_교육/문화용품_r,H1_기타_r,H2_기타_r,H3_기타_r,H4_기타_r,H1_디지털/가전_r,H2_디지털/가전_r,H3_디지털/가전_r,H4_디지털/가전_r,H1_명품_r,H2_명품_r,H3_명품_r,H4_명품_r,H1_신선식품_r,H2_신선식품_r,H3_신선식품_r,H4_신선식품_r,H1_의류_r,H2_의류_r,H3_의류_r,H4_의류_r,H1_일상용품_r,H2_일상용품_r,H3_일상용품_r,H4_일상용품_r,H1_전문스포츠/레저_r,H2_전문스포츠/레저_r,...,H1_전문스포츠/레저_app_r,H2_전문스포츠/레저_app_r,H3_전문스포츠/레저_app_r,H4_전문스포츠/레저_app_r,H1_패션잡화_app_r,H2_패션잡화_app_r,H3_패션잡화_app_r,H4_패션잡화_app_r,wdys_ratio_1,wdys_ratio_2,wdys_ratio_3,wdys_ratio_4,mor_ratio_1,mor_ratio_2,mor_ratio_3,mor_ratio_4,A_ratio_1,A_ratio_2,A_ratio_3,A_ratio_4,B_ratio_1,B_ratio_2,B_ratio_3,B_ratio_4,C_ratio_1,C_ratio_2,C_ratio_3,C_ratio_4,D_ratio_1,D_ratio_2,D_ratio_3,D_ratio_4,Frequency_1,Frequency_2,Frequency_3,Frequency_4,Recency_1,Recency_2,Recency_3,Recency_4
0,6012,1,4,1,1,2,7,5,7,7,8,7,9,10,4,4,4,4,8,4,4,9,4,4,4,4,1,1,1,1,10,10,10,10,5,5,4,8,6,7,...,10,8,2,2,7,5,9,3,4,2,1,2,10,9,8,6,9,9,9,8,4,4,4,5,3,6,5,3,5,5,5,5,10,6,8,6,5,8,8,8
1,12446,0,1,1,1,3,10,10,10,10,10,3,3,3,4,4,4,4,10,10,10,9,9,9,4,9,9,8,9,7,6,10,7,8,3,1,4,2,9,10,...,4,5,4,10,5,8,4,8,1,1,4,4,5,4,3,2,9,10,9,9,5,2,4,4,3,3,5,6,5,5,10,10,2,1,3,1,8,1,2,3
2,1909,1,1,1,1,1,9,9,9,9,7,3,6,6,7,4,4,4,8,8,4,4,10,4,10,4,9,9,10,10,9,9,9,8,2,3,2,2,10,10,...,8,5,7,6,5,7,8,7,9,9,10,8,2,1,2,5,8,8,8,7,6,6,6,6,3,6,3,3,5,5,5,5,5,3,4,6,3,5,2,1
3,9627,0,5,5,3,8,7,8,6,10,3,3,3,3,4,4,4,4,9,4,9,10,10,9,9,4,2,2,6,1,10,10,9,8,6,7,4,3,9,8,...,5,8,7,6,4,5,4,10,7,4,9,1,6,7,4,8,9,10,10,8,4,2,2,3,3,3,3,8,5,5,5,5,5,2,3,1,8,2,2,1
4,16297,1,1,1,1,2,7,6,5,7,9,10,10,10,4,8,4,8,7,4,8,8,4,4,9,10,10,10,9,8,7,7,8,8,3,4,8,8,6,2,...,10,2,4,2,6,7,8,5,6,6,3,9,10,10,9,9,6,6,6,7,7,7,7,7,3,3,3,5,5,5,5,5,10,10,9,7,8,8,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19080,19355,0,4,8,9,8,2,6,2,2,3,3,3,3,4,8,4,4,4,4,4,4,4,4,4,4,1,1,1,1,1,6,1,5,10,10,10,10,2,7,...,2,9,2,10,2,2,2,2,1,7,2,4,7,8,9,10,2,6,2,6,2,4,5,2,3,3,3,3,10,10,10,10,1,3,3,3,3,8,3,3
19081,18986,0,1,3,1,1,2,5,2,7,3,3,7,3,4,4,4,4,4,4,4,4,4,4,4,9,1,1,1,1,1,7,4,6,10,10,10,10,2,2,...,2,2,2,10,2,10,2,2,3,6,9,4,10,10,8,9,2,6,5,6,2,6,5,4,3,6,3,3,10,10,10,10,1,2,2,3,3,1,8,2
19082,6868,1,1,1,1,8,2,2,2,2,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,1,1,1,1,1,1,1,1,10,10,10,10,2,2,...,2,2,2,2,2,2,2,10,10,2,1,2,1,1,5,5,2,2,5,6,2,2,2,2,3,5,3,8,10,10,10,10,1,1,1,1,1,1,1,3
19083,18977,0,8,3,5,5,2,2,2,2,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,1,1,1,1,1,4,5,1,10,10,10,10,2,2,...,2,2,2,2,2,2,2,2,3,7,10,10,7,10,10,9,2,4,5,2,2,2,2,4,3,3,3,3,10,10,10,10,1,3,3,1,1,5,8,1


In [ ]:
abs_net_df = df1234
고객번호추가 = abs_net_df['고객번호']
target추가 = abs_net_df['target']
abs_net_df = abs_net_df.drop(['고객번호','target'],axis=1)

import pandas as pd
import numpy as np
changed_df = pd.DataFrame()
changed_df

group_col =[]
for i in range(int(len(abs_net_df.columns)/4)):
  group_col.append(list(abs_net_df.columns[i*4:i*4+4]))

columns = []
for cols in group_col:
  if cols[0][0]=='H':
    colname = cols[0][3:]
    columns.append(colname)
  else:
    colname = cols[0][:-2]
    columns.append(colname)

for index,cols in enumerate(group_col):
  get_cols = abs_net_df[cols]
  col1=get_cols.iloc[:,0]
  col2=get_cols.iloc[:,1]
  col3=get_cols.iloc[:,2]
  col4=get_cols.iloc[:,3]
  NetDiff = col4-col1
  AbsDiffsum = np.abs(col2-col1) + np.abs(col3-col2)+ np.abs(col4-col3)
  changed_df[f'{columns[index]}_Net'] = NetDiff
  changed_df[f'{columns[index]}_Abs'] = AbsDiffsum

changed_df['고객번호'] = 고객번호추가
changed_df['target'] = target추가



In [ ]:
changed_df.head()

,가공식품_r_Net,가공식품_r_Abs,가구/인테리어_r_Net,가구/인테리어_r_Abs,교육/문화용품_r_Net,교육/문화용품_r_Abs,기타_r_Net,기타_r_Abs,디지털/가전_r_Net,디지털/가전_r_Abs,명품_r_Net,명품_r_Abs,신선식품_r_Net,신선식품_r_Abs,의류_r_Net,의류_r_Abs,일상용품_r_Net,일상용품_r_Abs,전문스포츠/레저_r_Net,전문스포츠/레저_r_Abs,패션잡화_r_Net,패션잡화_r_Abs,객단가_Net,객단가_Abs,가공식품_구매금액_r_Net,가공식품_구매금액_r_Abs,가구/인테리어_구매금액_r_Net,가구/인테리어_구매금액_r_Abs,교육/문화용품_구매금액_r_Net,교육/문화용품_구매금액_r_Abs,기타_구매금액_r_Net,기타_구매금액_r_Abs,디지털/가전_구매금액_r_Net,디지털/가전_구매금액_r_Abs,명품_구매금액_r_Net,명품_구매금액_r_Abs,신선식품_구매금액_r_Net,신선식품_구매금액_r_Abs,의류_구매금액_r_Net,의류_구매금액_r_Abs,...,가공식품_app_r_Net,가공식품_app_r_Abs,가구/인테리어_app_r_Net,가구/인테리어_app_r_Abs,교육/문화용품_app_r_Net,교육/문화용품_app_r_Abs,기타_app_r_Net,기타_app_r_Abs,디지털/가전_app_r_Net,디지털/가전_app_r_Abs,명품_app_r_Net,명품_app_r_Abs,신선식품_app_r_Net,신선식품_app_r_Abs,의류_app_r_Net,의류_app_r_Abs,일상용품_app_r_Net,일상용품_app_r_Abs,전문스포츠/레저_app_r_Net,전문스포츠/레저_app_r_Abs,패션잡화_app_r_Net,패션잡화_app_r_Abs,wdys_ratio_Net,wdys_ratio_Abs,mor_ratio_Net,mor_ratio_Abs,A_ratio_Net,A_ratio_Abs,B_ratio_Net,B_ratio_Abs,C_ratio_Net,C_ratio_Abs,D_ratio_Net,D_ratio_Abs,Frequency_Net,Frequency_Abs,Recency_Net,Recency_Abs,고객번호,target
0,-2,4,0,4,2,4,0,0,1,9,0,0,0,0,0,0,3,5,-4,6,-2,8,0,0,0,0,1,1,0,2,0,0,1,7,0,0,0,0,0,0,...,-2,12,1,11,-1,5,0,0,0,10,0,0,8,10,-7,11,-3,15,-8,8,-4,12,-2,4,-4,4,-1,1,1,1,0,6,0,0,-4,8,3,3,6012,1
1,2,2,0,0,-7,7,0,0,-1,1,0,10,-2,4,2,8,-1,7,1,1,2,4,0,0,0,2,-3,5,-5,5,0,0,-2,4,0,10,-1,5,0,10,...,6,12,-2,8,-7,7,0,0,-1,5,2,10,-8,24,0,6,-3,13,6,8,3,11,3,3,-3,3,0,2,-1,5,3,3,5,5,-1,5,-5,9,12446,0
2,0,0,0,0,-1,7,-3,3,-4,4,-6,18,1,1,-1,1,0,2,-1,1,1,1,-1,3,1,5,1,1,0,8,-3,3,-4,8,-6,18,2,2,-1,3,...,-7,7,-2,8,2,10,-6,6,-4,8,-6,16,-5,19,-3,9,5,15,-2,6,2,4,-1,3,3,5,-1,1,0,0,0,6,0,0,1,5,-2,6,1909,1
3,3,7,3,7,0,0,0,0,1,11,-6,6,-1,9,-2,2,-3,5,0,2,-1,3,2,4,-2,2,4,12,0,0,0,0,2,10,-6,6,-1,5,-2,4,...,-8,8,5,7,0,0,0,0,2,10,-6,6,-8,12,5,7,-2,16,1,5,6,8,-6,16,2,8,-1,3,-1,3,5,5,0,0,-4,6,-7,7,9627,0
4,1,1,0,4,1,1,4,12,1,7,6,6,-2,2,1,1,5,5,-4,14,1,1,1,1,0,0,0,4,-1,1,3,9,1,7,6,6,-3,3,1,1,...,4,8,4,14,0,4,6,16,1,9,5,7,4,6,3,5,4,6,-8,12,-1,5,3,9,-1,1,1,1,0,0,2,2,0,0,-3,3,-5,5,16297,1


In [ ]:
import pickle5 as pickle
with open('/content/drive/MyDrive/5조/진짜진짜최종/2차데이터123분기.pkl', 'rb') as f:
    cust = pickle.load(f)

changed_df = pd.merge(changed_df,cust[['고객번호', '성별', '연령대', '거주지역']],on='고객번호', how='left')
changed_df.head()

,가공식품_r_Net,가공식품_r_Abs,가구/인테리어_r_Net,가구/인테리어_r_Abs,교육/문화용품_r_Net,교육/문화용품_r_Abs,기타_r_Net,기타_r_Abs,디지털/가전_r_Net,디지털/가전_r_Abs,명품_r_Net,명품_r_Abs,신선식품_r_Net,신선식품_r_Abs,의류_r_Net,의류_r_Abs,일상용품_r_Net,일상용품_r_Abs,전문스포츠/레저_r_Net,전문스포츠/레저_r_Abs,패션잡화_r_Net,패션잡화_r_Abs,객단가_Net,객단가_Abs,가공식품_구매금액_r_Net,가공식품_구매금액_r_Abs,가구/인테리어_구매금액_r_Net,가구/인테리어_구매금액_r_Abs,교육/문화용품_구매금액_r_Net,교육/문화용품_구매금액_r_Abs,기타_구매금액_r_Net,기타_구매금액_r_Abs,디지털/가전_구매금액_r_Net,디지털/가전_구매금액_r_Abs,명품_구매금액_r_Net,명품_구매금액_r_Abs,신선식품_구매금액_r_Net,신선식품_구매금액_r_Abs,의류_구매금액_r_Net,의류_구매금액_r_Abs,...,가구/인테리어_app_r_Abs,교육/문화용품_app_r_Net,교육/문화용품_app_r_Abs,기타_app_r_Net,기타_app_r_Abs,디지털/가전_app_r_Net,디지털/가전_app_r_Abs,명품_app_r_Net,명품_app_r_Abs,신선식품_app_r_Net,신선식품_app_r_Abs,의류_app_r_Net,의류_app_r_Abs,일상용품_app_r_Net,일상용품_app_r_Abs,전문스포츠/레저_app_r_Net,전문스포츠/레저_app_r_Abs,패션잡화_app_r_Net,패션잡화_app_r_Abs,wdys_ratio_Net,wdys_ratio_Abs,mor_ratio_Net,mor_ratio_Abs,A_ratio_Net,A_ratio_Abs,B_ratio_Net,B_ratio_Abs,C_ratio_Net,C_ratio_Abs,D_ratio_Net,D_ratio_Abs,Frequency_Net,Frequency_Abs,Recency_Net,Recency_Abs,고객번호,target,성별,연령대,거주지역
0,-2,4,0,4,2,4,0,0,1,9,0,0,0,0,0,0,3,5,-4,6,-2,8,0,0,0,0,1,1,0,2,0,0,1,7,0,0,0,0,0,0,...,11,-1,5,0,0,0,10,0,0,8,10,-7,11,-3,15,-8,8,-4,12,-2,4,-4,4,-1,1,1,1,0,6,0,0,-4,8,3,3,6012,1,F,40세~44세,서울 동작구
1,2,2,0,0,-7,7,0,0,-1,1,0,10,-2,4,2,8,-1,7,1,1,2,4,0,0,0,2,-3,5,-5,5,0,0,-2,4,0,10,-1,5,0,10,...,8,-7,7,0,0,-1,5,2,10,-8,24,0,6,-3,13,6,8,3,11,3,3,-3,3,0,2,-1,5,3,3,5,5,-1,5,-5,9,12446,0,F,45세~49세,서울 광진구
2,0,0,0,0,-1,7,-3,3,-4,4,-6,18,1,1,-1,1,0,2,-1,1,1,1,-1,3,1,5,1,1,0,8,-3,3,-4,8,-6,18,2,2,-1,3,...,8,2,10,-6,6,-4,8,-6,16,-5,19,-3,9,5,15,-2,6,2,4,-1,3,3,5,-1,1,0,0,0,6,0,0,1,5,-2,6,1909,1,F,50세~54세,경기 김포시
3,3,7,3,7,0,0,0,0,1,11,-6,6,-1,9,-2,2,-3,5,0,2,-1,3,2,4,-2,2,4,12,0,0,0,0,2,10,-6,6,-1,5,-2,4,...,7,0,0,0,0,2,10,-6,6,-8,12,5,7,-2,16,1,5,6,8,-6,16,2,8,-1,3,-1,3,5,5,0,0,-4,6,-7,7,9627,0,F,30세~34세,부산 기장
4,1,1,0,4,1,1,4,12,1,7,6,6,-2,2,1,1,5,5,-4,14,1,1,1,1,0,0,0,4,-1,1,3,9,1,7,6,6,-3,3,1,1,...,14,0,4,6,16,1,9,5,7,4,6,3,5,4,6,-8,12,-1,5,3,9,-1,1,1,1,0,0,2,2,0,0,-3,3,-5,5,16297,1,F,45세~49세,서울 도봉구


In [ ]:
import pickle5 as pickle
with open("/content/drive/MyDrive/5조/진짜진짜최종/4_최종추가_컬럼/df1234_멤버십.pkl", 'rb') as f:
    membership= pickle.load(f)
membership = membership.reset_index()
changed_df = pd.merge(changed_df,membership[['고객번호','abs_14']],on='고객번호', how='left')
changed_df['abs_14'].fillna(0,inplace=True)
changed_df = changed_df.rename(columns={'abs_14':'membership_abs'})
changed_df.head()

,가공식품_r_Net,가공식품_r_Abs,가구/인테리어_r_Net,가구/인테리어_r_Abs,교육/문화용품_r_Net,교육/문화용품_r_Abs,기타_r_Net,기타_r_Abs,디지털/가전_r_Net,디지털/가전_r_Abs,명품_r_Net,명품_r_Abs,신선식품_r_Net,신선식품_r_Abs,의류_r_Net,의류_r_Abs,일상용품_r_Net,일상용품_r_Abs,전문스포츠/레저_r_Net,전문스포츠/레저_r_Abs,패션잡화_r_Net,패션잡화_r_Abs,객단가_Net,객단가_Abs,가공식품_구매금액_r_Net,가공식품_구매금액_r_Abs,가구/인테리어_구매금액_r_Net,가구/인테리어_구매금액_r_Abs,교육/문화용품_구매금액_r_Net,교육/문화용품_구매금액_r_Abs,기타_구매금액_r_Net,기타_구매금액_r_Abs,디지털/가전_구매금액_r_Net,디지털/가전_구매금액_r_Abs,명품_구매금액_r_Net,명품_구매금액_r_Abs,신선식품_구매금액_r_Net,신선식품_구매금액_r_Abs,의류_구매금액_r_Net,의류_구매금액_r_Abs,...,교육/문화용품_app_r_Net,교육/문화용품_app_r_Abs,기타_app_r_Net,기타_app_r_Abs,디지털/가전_app_r_Net,디지털/가전_app_r_Abs,명품_app_r_Net,명품_app_r_Abs,신선식품_app_r_Net,신선식품_app_r_Abs,의류_app_r_Net,의류_app_r_Abs,일상용품_app_r_Net,일상용품_app_r_Abs,전문스포츠/레저_app_r_Net,전문스포츠/레저_app_r_Abs,패션잡화_app_r_Net,패션잡화_app_r_Abs,wdys_ratio_Net,wdys_ratio_Abs,mor_ratio_Net,mor_ratio_Abs,A_ratio_Net,A_ratio_Abs,B_ratio_Net,B_ratio_Abs,C_ratio_Net,C_ratio_Abs,D_ratio_Net,D_ratio_Abs,Frequency_Net,Frequency_Abs,Recency_Net,Recency_Abs,고객번호,target,성별,연령대,거주지역,membership_abs
0,-2,4,0,4,2,4,0,0,1,9,0,0,0,0,0,0,3,5,-4,6,-2,8,0,0,0,0,1,1,0,2,0,0,1,7,0,0,0,0,0,0,...,-1,5,0,0,0,10,0,0,8,10,-7,11,-3,15,-8,8,-4,12,-2,4,-4,4,-1,1,1,1,0,6,0,0,-4,8,3,3,6012,1,F,40세~44세,서울 동작구,0.0
1,2,2,0,0,-7,7,0,0,-1,1,0,10,-2,4,2,8,-1,7,1,1,2,4,0,0,0,2,-3,5,-5,5,0,0,-2,4,0,10,-1,5,0,10,...,-7,7,0,0,-1,5,2,10,-8,24,0,6,-3,13,6,8,3,11,3,3,-3,3,0,2,-1,5,3,3,5,5,-1,5,-5,9,12446,0,F,45세~49세,서울 광진구,0.0
2,0,0,0,0,-1,7,-3,3,-4,4,-6,18,1,1,-1,1,0,2,-1,1,1,1,-1,3,1,5,1,1,0,8,-3,3,-4,8,-6,18,2,2,-1,3,...,2,10,-6,6,-4,8,-6,16,-5,19,-3,9,5,15,-2,6,2,4,-1,3,3,5,-1,1,0,0,0,6,0,0,1,5,-2,6,1909,1,F,50세~54세,경기 김포시,0.0
3,3,7,3,7,0,0,0,0,1,11,-6,6,-1,9,-2,2,-3,5,0,2,-1,3,2,4,-2,2,4,12,0,0,0,0,2,10,-6,6,-1,5,-2,4,...,0,0,0,0,2,10,-6,6,-8,12,5,7,-2,16,1,5,6,8,-6,16,2,8,-1,3,-1,3,5,5,0,0,-4,6,-7,7,9627,0,F,30세~34세,부산 기장,0.0
4,1,1,0,4,1,1,4,12,1,7,6,6,-2,2,1,1,5,5,-4,14,1,1,1,1,0,0,0,4,-1,1,3,9,1,7,6,6,-3,3,1,1,...,0,4,6,16,1,9,5,7,4,6,3,5,4,6,-8,12,-1,5,3,9,-1,1,1,1,0,0,2,2,0,0,-3,3,-5,5,16297,1,F,45세~49세,서울 도봉구,0.0


In [ ]:
changed_df = changed_df[['성별', '연령대', '거주지역', 'membership_abs', '객단가_Net', '객단가_Abs',
       '가공식품_app_r_Net', '가공식품_app_r_Abs', '가구/인테리어_app_r_Net',
       '가구/인테리어_app_r_Abs', '교육/문화용품_app_r_Net', '교육/문화용품_app_r_Abs',
       '기타_app_r_Net', '기타_app_r_Abs', '디지털/가전_app_r_Net', '디지털/가전_app_r_Abs',
       '명품_app_r_Net', '명품_app_r_Abs', '신선식품_app_r_Net', '신선식품_app_r_Abs',
       '의류_app_r_Net', '의류_app_r_Abs', '일상용품_app_r_Net', '일상용품_app_r_Abs',
       '전문스포츠/레저_app_r_Net', '전문스포츠/레저_app_r_Abs', '패션잡화_app_r_Net',
       '패션잡화_app_r_Abs', 'wdys_ratio_Net', 'wdys_ratio_Abs', 'mor_ratio_Net',
       'mor_ratio_Abs', 'A_ratio_Net', 'A_ratio_Abs', 'B_ratio_Net',
       'B_ratio_Abs', 'C_ratio_Net', 'C_ratio_Abs', 'D_ratio_Net',
       'D_ratio_Abs', 'Frequency_Net', 'Frequency_Abs', 'Recency_Net',
       'Recency_Abs', 'target']]

In [ ]:
changed_df.head()

,고객번호,성별,연령대,거주지역,membership_abs,객단가_Net,객단가_Abs,가공식품_r_Net,가공식품_r_Abs,가구/인테리어_r_Net,가구/인테리어_r_Abs,교육/문화용품_r_Net,교육/문화용품_r_Abs,기타_r_Net,기타_r_Abs,디지털/가전_r_Net,디지털/가전_r_Abs,명품_r_Net,명품_r_Abs,신선식품_r_Net,신선식품_r_Abs,의류_r_Net,의류_r_Abs,일상용품_r_Net,일상용품_r_Abs,전문스포츠/레저_r_Net,전문스포츠/레저_r_Abs,패션잡화_r_Net,패션잡화_r_Abs,가공식품_구매금액_r_Net,가공식품_구매금액_r_Abs,가구/인테리어_구매금액_r_Net,가구/인테리어_구매금액_r_Abs,교육/문화용품_구매금액_r_Net,교육/문화용품_구매금액_r_Abs,기타_구매금액_r_Net,기타_구매금액_r_Abs,디지털/가전_구매금액_r_Net,디지털/가전_구매금액_r_Abs,명품_구매금액_r_Net,...,패션잡화_구매금액_r_Abs,가공식품_app_r_Net,가공식품_app_r_Abs,가구/인테리어_app_r_Net,가구/인테리어_app_r_Abs,교육/문화용품_app_r_Net,교육/문화용품_app_r_Abs,기타_app_r_Net,기타_app_r_Abs,디지털/가전_app_r_Net,디지털/가전_app_r_Abs,명품_app_r_Net,명품_app_r_Abs,신선식품_app_r_Net,신선식품_app_r_Abs,의류_app_r_Net,의류_app_r_Abs,일상용품_app_r_Net,일상용품_app_r_Abs,전문스포츠/레저_app_r_Net,전문스포츠/레저_app_r_Abs,패션잡화_app_r_Net,패션잡화_app_r_Abs,wdys_ratio_Net,wdys_ratio_Abs,mor_ratio_Net,mor_ratio_Abs,A_ratio_Net,A_ratio_Abs,B_ratio_Net,B_ratio_Abs,C_ratio_Net,C_ratio_Abs,D_ratio_Net,D_ratio_Abs,Frequency_Net,Frequency_Abs,Recency_Net,Recency_Abs,target
0,6012,F,40세~44세,서울 동작구,0.0,0,0,-2,4,0,4,2,4,0,0,1,9,0,0,0,0,0,0,3,5,-4,6,-2,8,0,0,1,1,0,2,0,0,1,7,0,...,2,-2,12,1,11,-1,5,0,0,0,10,0,0,8,10,-7,11,-3,15,-8,8,-4,12,-2,4,-4,4,-1,1,1,1,0,6,0,0,-4,8,3,3,1
1,12446,F,45세~49세,서울 광진구,0.0,0,0,2,2,0,0,-7,7,0,0,-1,1,0,10,-2,4,2,8,-1,7,1,1,2,4,0,2,-3,5,-5,5,0,0,-2,4,0,...,7,6,12,-2,8,-7,7,0,0,-1,5,2,10,-8,24,0,6,-3,13,6,8,3,11,3,3,-3,3,0,2,-1,5,3,3,5,5,-1,5,-5,9,0
2,1909,F,50세~54세,경기 김포시,0.0,-1,3,0,0,0,0,-1,7,-3,3,-4,4,-6,18,1,1,-1,1,0,2,-1,1,1,1,1,5,1,1,0,8,-3,3,-4,8,-6,...,5,-7,7,-2,8,2,10,-6,6,-4,8,-6,16,-5,19,-3,9,5,15,-2,6,2,4,-1,3,3,5,-1,1,0,0,0,6,0,0,1,5,-2,6,1
3,9627,F,30세~34세,부산 기장,0.0,2,4,3,7,3,7,0,0,0,0,1,11,-6,6,-1,9,-2,2,-3,5,0,2,-1,3,-2,2,4,12,0,0,0,0,2,10,-6,...,1,-8,8,5,7,0,0,0,0,2,10,-6,6,-8,12,5,7,-2,16,1,5,6,8,-6,16,2,8,-1,3,-1,3,5,5,0,0,-4,6,-7,7,0
4,16297,F,45세~49세,서울 도봉구,0.0,1,1,1,1,0,4,1,1,4,12,1,7,6,6,-2,2,1,1,5,5,-4,14,1,1,0,0,0,4,-1,1,3,9,1,7,6,...,3,4,8,4,14,0,4,6,16,1,9,5,7,4,6,3,5,4,6,-8,12,-1,5,3,9,-1,1,1,1,0,0,2,2,0,0,-3,3,-5,5,1


In [ ]:
from sklearn.preprocessing import LabelEncoder
                                                                                                                                                        
train_df = changed_df.drop('고객번호',axis = 1)

encoder = LabelEncoder()


train_df['성별'] = encoder.fit_transform(train_df['성별'])
train_df['연령대'] = encoder.fit_transform(train_df['연령대'])
train_df['거주지역']=train_df['거주지역'].fillna('불명')
train_df['거주지역'] = encoder.fit_transform(train_df['거주지역'])

In [ ]:
with open("/content/drive/MyDrive/5조/진짜멘탈터지고 최종/df_ml_1234.pkl",'wb') as test_save:
  pickle.dump(train_df, test_save)